In [114]:
import mysql.connector

config = {
    "user": "root",
    "password": "root",
    "host": "127.0.0.1",
    "database": "senior_design"
}
connection = mysql.connector.connect(**config)
cursor = connection.cursor(buffered=True)

In [115]:
def get_all_reviews(asin):
    query = "SELECT review_text FROM review WHERE asin = '{}'".format(asin)
    cursor.execute(query)
    reviews = []
    for (review_text) in cursor:
        reviews.append(review_text[0])
    print("# reviews: {}".format(len(reviews)))
    return reviews

reviews = get_all_reviews("B002P3F5D2")

# reviews: 99


In [116]:
print(reviews[0])

I bought this camcorder in spite of all the negative reviews i read, because I wanted an external mic input for better sound quality in my videos, and most pocket camcorders have fazed out the external mic option. So this camera is from a couple years back I guess, but I got it brand new in the box, not a refurb, for 80 bucks Shoots in hi-def and widescreen, and the videos come out beautiful, even in regular hi-def mode. There is also an HD+ mode, but you only get half the recording time, about an hour. No kidding, this thing beats my FlipCam 8 ways to Sunday. I did have to spring an extra 25 or so for a tripod, since the VADO is pretty slim and won't stand by itself, but having a tripod only increases my options. It's got a rechargeable battery that will run about an hour and a half, two hours tops. One thing, the price on this thing fluctuates wildly. I got it for 80, when I looked the next day it was a hundred, and the day after that, $120. This from the same vendor. Bottom line, I'

In [117]:
positive_lexicon = {"good", "great", "better", "excellent", "best", "easy", "nice", "simple", "clear", "strong", 
                    "perfect", "comfortable", "friendly", "solid", "precise", "awesome", "amazing", "bright", "vibrant",
                    "fantastic", "vibrant", "realistic", "stunning", "superior", "super", "rich", "exceptional",
                    "impressive", "ideal"}
negative_lexicon = {"poor", "old", "bad", "weak", "annoying", "defective", "horrible", "buggy", "worst", "mediocre",
                    "difficult", "unstable", "inferior", "lousy", "complicated", "useless", "unreliable", "sloppy",
                    "strange", "weird", "malfunctioning", "miserable", "terrible", "misleading"}

In [118]:
from nltk.parse.corenlp import CoreNLPDependencyParser
# Start the CoreNLP server with:
# java -mx4g -cp "./CoreNLP/*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

nlp = CoreNLPDependencyParser(url="http://localhost:9000")

In [119]:
from collections import defaultdict

# output: all features, expanded opinion lexicon
def extract_features_opinions(reviews):
    opinions = positive_lexicon.union(negative_lexicon)
    features = set()
    OF_dict = {}
    FO_dict = {}
    OO_dict = defaultdict(list)
    FF_dict = defaultdict(list)
    sentences = []
    review_indices = []
    for i, review in enumerate(reviews):
        #print(review)
        parse = nlp.parse_text(review)
        parses.append(parse)
        for sentence in parse:
            extracted_sentence = []
            sentences.append(extracted_sentence)
            review_indices.append(i)
            for (gov, gov_pos), dependency, (dep, dep_pos) in sentence.triples():
                if not gov.isalpha() or not dep.isalpha():
                    continue
                gov = gov.lower()
                dep = dep.lower()
                if dependency == "nsubj" and dep_pos == "NN":
                    OF_dict[gov] = dep
                    FO_dict[dep] = gov
                elif dependency == "amod" and gov_pos == "NN":
                    OF_dict[dep] = gov
                    FO_dict[gov] = dep
                elif dependency == "conj":
                    if gov_pos == "JJ" and dep_pos == "JJ":
                        OO_dict[gov].append(dep)
                        OO_dict[dep].append(gov)
                    elif gov_pos == "NN" and dep_pos == "NN":
                        FF_dict[gov].append(dep)
                        FF_dict[dep].append(gov)
                extracted_sentence.append(((gov, gov_pos), dependency, (dep, dep_pos)))
    i = 0
    while (True):
        print("Iteration: {}".format(i))
        i += 1
        new_opinions = set()
        new_features = set()
        for opinion, feature in OF_dict.items():
            if opinion in opinions and feature not in features:
                new_features.add(feature)
        for opinion1, related in OO_dict.items():
            if opinion1 in opinions:
                for opinion in related:
                    if opinion not in opinions:
                        new_opinions.add(opinion)
        for feature, opinion in FO_dict.items():
            if feature in features and opinion not in opinions:
                new_opinions.add(opinion)
        for feature1, related in FF_dict.items():
            if feature1 in features:
                for feature in related:
                    if feature not in features:
                        new_features.add(feature)
        opinions = opinions.union(new_opinions)
        features = features.union(new_features)
        if len(new_opinions) == 0 and len(new_features) == 0:
            break
    return features, opinions, sentences, review_indices
        
features, opinions, sentences, review_indices = extract_features_opinions(reviews)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8


In [120]:
print(features)
print(opinions.difference(positive_lexicon.union(negative_lexicon)))

set([u'point', u'feel', u'storage', u'image', u'date', u'gb', u'shake', u'fidelity', u'quality', u'unit', u'size', u'slot', u'use', u'footage', u'tv', u'support', u'show', u'connector', u'pc', u'bike', u'hd', u'lot', u'sd', u'sale', u'kid', u'sensitivity', u'case', u'option', u'purchase', u'brand', u'tripod', u'phone', u'hdtv', u'facebook', u'jack', u'color', u'sound', u'television', u'portability', u'aac', u'button', u'instruction', u'flip', u'recording', u'record', u'convenience', u'mode', u'charger', u'fun', u'design', u'problem', u'setup', u'def', u'control', u'weight', u'process', u'twitter', u'zoom', u'daylight', u'computer', u'video', u'something', u'file', u'operation', u'arm', u'battery', u'capture', u'microphone', u'bag', u'sun', u'review', u'novice', u'switch', u'port', u'state', u'playback', u'performance', u'sensor', u'screen', u'accuracy', u'cord', u'picture', u'product', u'finish', u'ability', u'power', u'camera', u'price', u'lens', u'lighting', u'device', u'card', u'exp

In [121]:
def process_reviews(features, opinions, sentences, review_indices):
    FO_dict = defaultdict(list)
    for i, sentence in enumerate(sentences):
        phrase_dict = {}
        FO_dict_sentence = defaultdict(list)
        for (gov, gov_pos), dependency, (dep, dep_pos) in sentence:
            if not gov.isalpha() or not dep.isalpha():
                continue
            gov = gov.lower()
            dep = dep.lower()
            if dependency == "nsubj" and dep in features:
                FO_dict_sentence[dep].append(gov)
                print("nsubj: {} -> {}, review #{}".format(gov, dep, review_indices[i]))
            elif dependency == "amod" and gov in features:
                FO_dict_sentence[gov].append(dep)
                print("amod: {} -> {}, review #{}".format(dep, gov, review_indices[i]))
            elif dependency == "compound" and gov in features:
                phrase_dict[gov] = dep + " " + gov
        for feature, opinions in FO_dict_sentence.items():
            if feature in phrase_dict:
                FO_dict[phrase_dict[feature]] += opinions
            else:
                FO_dict[feature] += opinions
    return FO_dict

FO_dict = process_reviews(features, opinions, sentences, review_indices)

amod: better -> quality, review #0
amod: sound -> quality, review #0
amod: external -> option, review #0
amod: mic -> option, review #0
nsubj: years -> camera, review #0
nsubj: shoots -> brand, review #0
amod: new -> brand, review #0
amod: regular -> mode, review #0
nsubj: is -> hd, review #0
nsubj: beats -> thing, review #0
nsubj: increases -> tripod, review #0
amod: rechargeable -> battery, review #0
nsubj: fluctuates -> price, review #0
nsubj: charge -> thing, review #0
amod: simple -> instruction, review #0
amod: beautiful -> video, review #0
nsubj: run -> option, review #0
amod: next -> option, review #0
amod: cheapest -> option, review #0
nsubj: better -> quality, review #1
amod: little -> camcorder, review #1
nsubj: ago -> product, review #1
nsubj: excellent -> quality, review #1
amod: usb -> connector, review #1
nsubj: user -> flip, review #2
nsubj: superior -> picture, review #2
amod: common -> problem, review #3
nsubj: needs -> battery, review #3
nsubj: jumpy -> video, review

In [124]:
print(reviews[32])

I owned the first generation of this camera, so when the lcd broke on the other one after an accident, I was pretty confident that I would love this one as well. Not so.The worst problem is the addition of the touch sensitive controls. It's very easy to accidentally push any of the buttons. I'd read about this in reviews before I got the camera, but it ended up being way worse of a problem than I expected. The only actual button on the back of the camera is the record button, and it must be way too easy to press as well, because in the short time that I owned it I recorded well over 3 hours of footage from my pocket. This even after I turned it off and disabled the quick record option. Call me old fashioned, but I think you should be able to put a "pocket camera" in your pocket. Also, I never had this problem with the first generation. Let me also say that I was really disappointed that they cut the amount memory in half from the first generation of this camera, and still without addin

In [123]:
import pprint

pp = pprint.PrettyPrinter()
pp.pprint(sorted(FO_dict.iteritems(), key=lambda (k,v): len(v), reverse=True))

[(u'camera',
  [u'years',
   u'handy',
   u'little',
   u'shakes',
   u'accumulate',
   u'seems',
   u'worked',
   u'flip',
   u'does',
   u'nice',
   u'little',
   u'great',
   u'little',
   u'works',
   u'goes',
   u'little',
   u'video',
   u'size',
   u'compact',
   u'digital',
   u'functions',
   u'turn',
   u'do',
   u'quality',
   u'outlived',
   u'one',
   u'best',
   u'dedicated',
   u'digital',
   u'comes',
   u'year',
   u'old',
   u'digital',
   u'go',
   u'digital',
   u'works',
   u'turn',
   u'fantastic',
   u'basic',
   u'inexpensive',
   u'versatile',
   u'external',
   u'mic',
   u'digital',
   u'easier',
   u'mounts',
   u'preferred',
   u'try',
   u'useless',
   u'work']),
 (u'quality',
  [u'better',
   u'sound',
   u'better',
   u'best',
   u'better',
   u'photo',
   u'is',
   u'exceptional',
   u'sound',
   u'video',
   u'is',
   u'sound',
   u'video',
   u'one',
   u'video',
   u'great',
   u'video',
   u'terrible',
   u'good',
   u'outstanding',
   u'great',
   